In [1]:
%load_ext Cython

In [2]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
    
from modules.io_modules import read_timestamp
from modules.alfa_rossi_preprocesamiento import corrige_roll_over

## Se leen los tiempos de llegada de los pulsos

In [3]:
nombres = [                                                                    
           '../datos/medicion04.a.inter.D1.bin',                                
           '../datos/medicion04.a.inter.D2.bin',
]                                                                   
                                                        
tiempos_cro = []
headers = []
for nombre in nombres:
    _tiempo_cro, _header = read_timestamp(nombre, common_time=True)
    tiempos_cro.append(_tiempo_cro)
    headers.append(_header)
    print(f"Primer dato de {nombre}: {_tiempo_cro[0]}\n")

tiempos = corrige_roll_over(tiempos_cro)


El archivo tiene encabezado
--------------------------------------------------
Primer dato de ../datos/medicion04.a.inter.D1.bin: 238291

El archivo tiene encabezado
--------------------------------------------------
Primer dato de ../datos/medicion04.a.inter.D2.bin: 244967

Corrigiendo roll-over del archivo [0]
Tipo de dato inicial: >u4
Tipo de dato final uint64
--------------------------------------------------
Corrigiendo roll-over del archivo [1]
Tipo de dato inicial: >u4
Tipo de dato final uint64
--------------------------------------------------


## Se definen las funciones para calcular las coincidencias

In [4]:
%%cython
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def calcula_coincidencias_double(double[:] tiempo_1, double[:] tiempo_2, double time_resolution, double delay):
    cdef int N1, N2, Nc, p, i, j
    N1 = tiempo_1.size
    N2 = tiempo_2.size
    Nc = 0
    p = 0
    for i in range(N1):
        for j in range(p, N2):
            if tiempo_2[j] + time_resolution > tiempo_1[i] + delay:
                if tiempo_2[j] < tiempo_1[i] + delay + time_resolution:
                    Nc += 1
                    p += 1
                    break
    return Nc

@cython.boundscheck(False)
@cython.wraparound(False)
def calcula_coincidencias_uint64(unsigned long long[:] tiempo_1, unsigned long long[:] tiempo_2, unsigned long long time_resolution, unsigned long long delay):
    cdef int N1, N2, Nc, p, i, j
    N1 = tiempo_1.size
    N2 = tiempo_2.size
    Nc = 0
    p = 0
    for i in range(N1):
        for j in range(p, N2):
            if tiempo_2[j] + time_resolution > tiempo_1[i] + delay:
                if tiempo_2[j] < tiempo_1[i] + delay + time_resolution:
                    Nc += 1
                    p += 1
                    break
    return Nc

In [5]:
def calcula_coincidencias(tiempo_1, tiempo_2, time_resolution, delay=0):
    """
    Calcula el número de coincidencias entre las señaels tiempo_1 y tiempo_2.
    
    Se suma un delay a tiempo_1. Todos los parámetros de entrada deben tener las mismas unidades,
    ya sea en segundos o en número de pulsos de reloj.
    
    Se considera coincidencia cuando un pulso de tiempo_1 está a menos de time_resolution de otro pulso
    de tiempo_2.
    
    Esta función es un wrapper de funciones pre-compiladas en cython, se deben respetar los tipos de datos
    para los vectores temporales (float64 o uint64).
    
    Parámetros
    ----------
        tiempo_1, tiempo_2: ndarray de float64 o uint64
            Tiempos de cada señal
        time_resolution: 
            Ventana temporal para las coincidencias
        delay:
            Delay que se le suma a tiempo_1
    
    Resultados
    ----------
        Nc: 
            Número de coincidencias
    
    """
    if tiempo_1.dtype == "float64":
        return calcula_coincidencias_double(tiempo_1, tiempo_2, time_resolution, delay)
    elif tiempo_1.dtype == "uint64":
        return calcula_coincidencias_uint64(tiempo_1, tiempo_2, time_resolution, delay)
    else:
        raise TypeError("El tipo de datos debe ser float64 (float, np.double) o uint64 (np.uint64, np.ulonglong)")

In [6]:
def calcula_coincidencias_lento(times_1, times_2, time_resolution, delay):
    times_2_d = times_2 + delay
    N1 = times_1.size
    N2 = times_2_d.size
    Nc = 0
    p = 0
    for i in range(N1):
        for j in range(p, N2):
            if times_2_d[j] > times_1[i] - time_resolution:
                if times_2_d[j] < times_1[i] + time_resolution:
                    Nc += 1
                    p += 1
                    break
    return Nc

## Prueba de las funciones

Pruebo con una misma señal con delay. Si el delay es más grande que la ventana, no debe contar coincidencias. De lo contrario se tienen tantas coincidencias como pulsos detectados.

In [7]:
# Selecciono una parte de los datos
dato = tiempos[0][0:1000]
# convierto datos a float64 
dato_float = np.asarray(dato, dtype=float)

In [8]:
#%%timeit
# Time resolution
tr = 9
# Delay
delay = 8
# Prueba cuando los datos son uint64
r_int = calcula_coincidencias(dato, dato, tr, delay)
# Prueba cuando los datos son float64
r_dbl = calcula_coincidencias(dato_float, dato_float, tr, delay)
print(r_int, r_dbl)

1000 1000


In [9]:
#%%timeit
# Puede tardar mucho si la cantidad de datos es > 1000
calcula_coincidencias_lento(dato, dato, tr, delay)

1000